# WGAN

- **DCGAN의 문제점:** 
    1) discriminator와 generator 간의 균형을 유지하며 학습하기 어려움
    2) 학습이 완료된 이후에도 mode dropping이 발생
- **원인:** discriminator가 선생님 역할을 충분히 해주지 못해 모델이 최적점까지 학습되지 못했기 때문


- **Wasserstein GAN의 차별점:**
    1) discriminator 대신 새로 정의한 critic을 사용. discriminator는 Real/Fake를 판별하기 위해 'sigmoid'를 사용, output은 예측 확률값
    2) critic은 EM(Earth Mover) distance로 부터 얻은 scalar값을 이용
    3) EM distance는 확률분포 간의 거리를 측정하는 척도 (기존 척도인 KL divergence는 매우 strict하기 때문에 비연속적인 경우가 있어 학습에 어려움이 있음)

# MNIST

In [13]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [14]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input
from tensorflow.keras.optimizers import RMSprop
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
from keras.models import load_model

import numpy as np
import argparse
import matplotlib.pyplot as plt

from lib import gan
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [15]:
def wasserstein_loss(y_label, y_pred):
    return -K.mean(y_label * y_pred)

In [16]:
def build_and_train_models():
    (x_train, _), (_, _) = mnist.load_data()
    
    # Reshape & Normalize
    image_size = x_train.shape[1]
    x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
    x_train = x_train.astype('float32')/255
    
    model_name = 'wgan_mnist'
    
    # Additional wgan params
    n_critic = 5
    clip_value = 0.01
    
    # Network Params
    latent_size = 100 # z-vector dimension
    batch_size = 64
    train_steps = 40000
    lr = 5e-5 # DCGAN: 2e-4
    # decay = 6e-8 # decays the lerning rate over time; DCGAN
    input_shape = (image_size, image_size, 1)

    # In Keras 2.11.0, 'decay' argument changed to 'weight_decay'
    # Discriminator model
    inputs = keras.Input(shape=input_shape, name='discriminator_input')
    
    discriminator = gan.discriminator(inputs, activation='linear')
    # discriminator = build_discriminator(inputs)
    optimizer = RMSprop(learning_rate=lr)
    discriminator.compile(loss=wasserstein_loss,
                          optimizer=optimizer,
                          metrics=['accuracy']
                         )
    discriminator.summary()
    
    # Generator model
    input_shape = (latent_size, )
    inputs = keras.Input(shape=input_shape, name='z_input')
    generator = gan.generator(inputs, image_size)
    generator.summary()
    
    # Adversarial model
    # optimizer = keras.optimizers.RMSprop(learning_rate=lr*0.5, decay=decay*0.5,)
    discriminator.trainable = False
    ## Adversarial = Generator + Discriminator
    adversarial = Model(inputs, discriminator(generator(inputs)), name=model_name)
    adversarial.compile(loss=wasserstein_loss,
                        optimizer=optimizer,
                        metrics=['accuracy']
                       )
    adversarial.summary()
    
    models = (generator, discriminator, adversarial)
    params = (batch_size, latent_size, n_critic, clip_value, train_steps, model_name)
    train(models, x_train, params)

In [17]:
def train(models, x_train, params):
    # Discriminator와 Adversarial Model을 배치 단위로 번갈아 훈련
    ## Discriminator는 제대로 레이블이 붙은 진짜와 가짜 이미지를 가지고 훈련
    ## Adversarial는 진짜인 척하는 가짜 이미지로 훈련
    
    # GAN Model
    generator, discriminator, adversarial = models
    
    # Network Params
    # batch_size, latent_size, train_steps, model_name = params
    batch_size, latent_size, n_critic, clip_value, train_steps, model_name = params
    
    # Save Generator Images every 500 epochs
    save_interval = 500
    
    # Noise_Input
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, latent_size]) # 16 x 100
    
    train_size = x_train.shape[0]
    real_labels = np.ones((batch_size, 1))
    for i in range(train_steps): # train_steps: 40,000
        
        # First - Discriminator Train
        # train discriminator n_critic times
        loss = 0
        acc = 0
        for _ in range(n_critic):
            # 1 batch of real (label=1.0) and fake images (label = -1.0)
            # randomly pick real images from dataset
            rand_indexes = np.random.randint(0, train_size, size=batch_size)
            real_images = x_train[rand_indexes]
            # generate fake images from noise using generator
            # generate noise using uniform distribution
            noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])
            fake_images = generator.predict(noise)
        
            # train the discriminator network
            # real data label=1, fake data label=-1
            # instead of 1 combined batch of real and fake images,
            # train with 1 batch of real data first, then 1 batch of fake images.
            # this tweak prevents the gradient from vanishing due to opposite signs of real and fake data labels
            # and small magnitude of weights due to clipping.
            real_loss, real_acc = discriminator.train_on_batch(real_images, real_labels)
            fake_loss, fake_acc = discriminator.train_on_batch(fake_images, -real_labels)
            
            # accumulate average loss and accuracy
            loss += 0.5 * (real_loss + fake_loss)
            acc += 0.5 * (real_acc + fake_acc)
            
            # clip discriminator weights to satisfy Lipschitz constraint
            for layer in discriminator.layers:
                weights = layer.get_weights()
                weights = [np.clip(weight, -clip_value, clip_value) for weight in weights]
                layer.set_weights(weights)
        
        # average loss and accuracy per n_critic training iterations
        loss /= n_critic
        acc /= n_critic
        log = "%d: [discriminator loss: %f, acc: %f]" %(i, loss, acc)
        
        # Second - Adversarial Train
        # train the adversarial network for 1 batch
        # 1 batch of fake images with label=1.0
        # since the discriminator weights are frozen in adversarial network
        # only the generator is trained
        # generate noise using uniform distribution
        noise = np.random.uniform(-1.0, 1.0, size=[batch_size, latent_size])

        # train the adversarial network
        # note that unlike in discriminator training,
        # we do not save the fake images in a variable
        # the fake images go to the discriminator input of the adversarial
        # for classification
        # fake images are labelled as real
        # log the loss and accuracy
        loss, acc = adversarial.train_on_batch(noise, real_labels)
        log = "%s: [adversarial loss: %f, acc: %f]" %(log, loss, acc)
        print(log)
        
        # Show generator images per 500 epochs
        if (i+1) % save_interval == 0: # 500
            if (i+1) == train_steps: # 40,000
                show = True
            else:
                show = False
                
            # Make generator images per 500 epochs
            gan.plot_images(generator,
                            noise_input=noise_input,
                            show=show,
                            step=(i+1),
                            model_name=model_name
                           )
    
    # save the model after training the generator
    # the trained generator can be reloaded for future MNIST digit generation
    generator.save(model_name + ".h5")

In [ ]:
if __name__=='__main__':
    build_and_train_models()

"""
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    help_ = "Load generator h5 model with trained weights"
    parser.add_argument("-g", "--generator", help=help_)
    args = parser.parse_args() # args에 위 내용 저장
    if args.generator:
        generator = load_model(args.generator)
        gan.test_generator(generator)
    else:
        build_and_train_models()
"""

linear
Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 discriminator_input (InputL  [(None, 28, 28, 1)]      0         
 ayer)                                                           
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 14, 14, 32)        832       
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 14, 14, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 64)          51264     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 7, 7, 64)          0         
                                              

2023-02-25 11:42:10.439866: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-25 11:42:10.497428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-25 11:42:10.786219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-02-25 11:42:11.891916: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


0: [discriminator loss: 0.014383, acc: 0.000000]: [adversarial loss: 0.000075, acc: 0.000000]
1: [discriminator loss: -0.000138, acc: 0.000000]: [adversarial loss: 0.000468, acc: 0.000000]
2: [discriminator loss: -0.000373, acc: 0.000000]: [adversarial loss: 0.001268, acc: 0.000000]
3: [discriminator loss: -0.000817, acc: 0.000000]: [adversarial loss: 0.002934, acc: 0.000000]
4: [discriminator loss: -0.001753, acc: 0.000000]: [adversarial loss: 0.006834, acc: 0.000000]
5: [discriminator loss: -0.003969, acc: 0.000000]: [adversarial loss: 0.015249, acc: 0.000000]
6: [discriminator loss: -0.009118, acc: 0.000000]: [adversarial loss: 0.031950, acc: 0.000000]
7: [discriminator loss: -0.019520, acc: 0.000000]: [adversarial loss: 0.059485, acc: 0.000000]
8: [discriminator loss: -0.036076, acc: 0.000000]: [adversarial loss: 0.099503, acc: 0.000000]
9: [discriminator loss: -0.062249, acc: 0.000000]: [adversarial loss: 0.151666, acc: 0.000000]
10: [discriminator loss: -0.099341, acc: 0.000000]:

85: [discriminator loss: -111.670599, acc: 0.020313]: [adversarial loss: 119.166801, acc: 0.000000]
86: [discriminator loss: -113.167386, acc: 0.017188]: [adversarial loss: 121.482750, acc: 0.000000]
87: [discriminator loss: -112.297404, acc: 0.010937]: [adversarial loss: 124.198288, acc: 0.000000]
88: [discriminator loss: -112.332230, acc: 0.026562]: [adversarial loss: 126.304848, acc: 0.000000]
89: [discriminator loss: -116.091599, acc: 0.010937]: [adversarial loss: 128.882660, acc: 0.000000]
90: [discriminator loss: -114.014706, acc: 0.006250]: [adversarial loss: 131.152405, acc: 0.000000]
91: [discriminator loss: -116.830572, acc: 0.010937]: [adversarial loss: 133.214569, acc: 0.000000]
92: [discriminator loss: -115.388780, acc: 0.010937]: [adversarial loss: 135.701996, acc: 0.000000]
93: [discriminator loss: -113.783231, acc: 0.012500]: [adversarial loss: 137.468292, acc: 0.000000]
94: [discriminator loss: -118.071098, acc: 0.007812]: [adversarial loss: 140.040680, acc: 0.000000]


167: [discriminator loss: -17.020981, acc: 0.004687]: [adversarial loss: -41.529003, acc: 1.000000]
168: [discriminator loss: -17.326106, acc: 0.004687]: [adversarial loss: -42.000591, acc: 1.000000]
169: [discriminator loss: -21.000203, acc: 0.003125]: [adversarial loss: -42.344635, acc: 1.000000]
170: [discriminator loss: -13.678764, acc: 0.006250]: [adversarial loss: -42.775761, acc: 1.000000]
171: [discriminator loss: -14.898647, acc: 0.003125]: [adversarial loss: -43.172474, acc: 1.000000]
172: [discriminator loss: -11.830434, acc: 0.006250]: [adversarial loss: -43.508213, acc: 1.000000]
173: [discriminator loss: -11.303897, acc: 0.006250]: [adversarial loss: -43.836304, acc: 1.000000]
174: [discriminator loss: -8.771138, acc: 0.007812]: [adversarial loss: -44.123730, acc: 1.000000]
175: [discriminator loss: -11.758958, acc: 0.012500]: [adversarial loss: -44.336861, acc: 1.000000]
176: [discriminator loss: -7.967792, acc: 0.015625]: [adversarial loss: -44.598228, acc: 1.000000]
17

...

24398: [discriminator loss: -0.004050, acc: 0.000000]: [adversarial loss: 0.005067, acc: 0.000000]
24399: [discriminator loss: -0.004866, acc: 0.000000]: [adversarial loss: 0.003591, acc: 0.000000]
24400: [discriminator loss: -0.004642, acc: 0.000000]: [adversarial loss: -0.009818, acc: 0.000000]
24401: [discriminator loss: -0.001289, acc: 0.000000]: [adversarial loss: -0.005839, acc: 0.000000]
24402: [discriminator loss: -0.005909, acc: 0.000000]: [adversarial loss: -0.009408, acc: 0.000000]
24403: [discriminator loss: -0.003204, acc: 0.000000]: [adversarial loss: -0.005955, acc: 0.000000]
24404: [discriminator loss: -0.004420, acc: 0.000000]: [adversarial loss: -0.013386, acc: 0.000000]
24405: [discriminator loss: -0.004323, acc: 0.000000]: [adversarial loss: -0.010427, acc: 0.000000]
24406: [discriminator loss: -0.002574, acc: 0.000000]: [adversarial loss: -0.013363, acc: 0.000000]
24407: [discriminator loss: -0.001539, acc: 0.000000]: [adversarial loss: -0.019365, acc: 0.000000]
24

24480: [discriminator loss: -0.002968, acc: 0.000000]: [adversarial loss: -0.013890, acc: 0.000000]
24481: [discriminator loss: -0.001009, acc: 0.000000]: [adversarial loss: 0.001752, acc: 0.000000]
24482: [discriminator loss: -0.003946, acc: 0.000000]: [adversarial loss: -0.006902, acc: 0.000000]
24483: [discriminator loss: 0.001754, acc: 0.000000]: [adversarial loss: -0.012113, acc: 0.000000]
24484: [discriminator loss: -0.002997, acc: 0.000000]: [adversarial loss: -0.011635, acc: 0.000000]
24485: [discriminator loss: -0.004436, acc: 0.000000]: [adversarial loss: -0.009763, acc: 0.000000]
24486: [discriminator loss: -0.002925, acc: 0.000000]: [adversarial loss: -0.006736, acc: 0.000000]
24487: [discriminator loss: -0.001745, acc: 0.000000]: [adversarial loss: -0.015364, acc: 0.000000]
24488: [discriminator loss: -0.000904, acc: 0.000000]: [adversarial loss: -0.015297, acc: 0.000000]
24489: [discriminator loss: -0.001232, acc: 0.000000]: [adversarial loss: -0.010290, acc: 0.000000]
24

24563: [discriminator loss: -0.004634, acc: 0.000000]: [adversarial loss: -0.021277, acc: 0.000000]
24564: [discriminator loss: -0.011191, acc: 0.000000]: [adversarial loss: -0.016094, acc: 0.000000]
24565: [discriminator loss: -0.001664, acc: 0.000000]: [adversarial loss: -0.040690, acc: 0.000000]
24566: [discriminator loss: 0.001523, acc: 0.000000]: [adversarial loss: -0.020397, acc: 0.000000]
24567: [discriminator loss: -0.001680, acc: 0.000000]: [adversarial loss: -0.016412, acc: 0.000000]
24568: [discriminator loss: -0.002376, acc: 0.000000]: [adversarial loss: -0.016081, acc: 0.000000]
24569: [discriminator loss: -0.006232, acc: 0.000000]: [adversarial loss: -0.018194, acc: 0.000000]
24570: [discriminator loss: -0.000727, acc: 0.000000]: [adversarial loss: -0.013427, acc: 0.000000]
24571: [discriminator loss: -0.002576, acc: 0.000000]: [adversarial loss: -0.002720, acc: 0.000000]
24572: [discriminator loss: -0.004895, acc: 0.000000]: [adversarial loss: -0.013155, acc: 0.000000]
2

24646: [discriminator loss: -0.002435, acc: 0.000000]: [adversarial loss: -0.012434, acc: 0.000000]
24647: [discriminator loss: -0.003076, acc: 0.000000]: [adversarial loss: -0.010741, acc: 0.000000]
24648: [discriminator loss: -0.003705, acc: 0.000000]: [adversarial loss: -0.007151, acc: 0.000000]
24649: [discriminator loss: -0.003800, acc: 0.000000]: [adversarial loss: -0.005805, acc: 0.000000]
24650: [discriminator loss: -0.003499, acc: 0.000000]: [adversarial loss: -0.025977, acc: 0.000000]
24651: [discriminator loss: -0.004293, acc: 0.000000]: [adversarial loss: -0.018919, acc: 0.000000]
24652: [discriminator loss: -0.000538, acc: 0.000000]: [adversarial loss: -0.005993, acc: 0.000000]
24653: [discriminator loss: -0.002217, acc: 0.000000]: [adversarial loss: -0.019403, acc: 0.000000]
24654: [discriminator loss: -0.003411, acc: 0.000000]: [adversarial loss: 0.000145, acc: 0.000000]
24655: [discriminator loss: -0.004196, acc: 0.000000]: [adversarial loss: 0.000272, acc: 0.000000]
24

24729: [discriminator loss: -0.002727, acc: 0.000000]: [adversarial loss: -0.006313, acc: 0.000000]
24730: [discriminator loss: -0.001666, acc: 0.000000]: [adversarial loss: -0.010050, acc: 0.000000]
24731: [discriminator loss: -0.002300, acc: 0.000000]: [adversarial loss: -0.011010, acc: 0.000000]
24732: [discriminator loss: -0.003650, acc: 0.000000]: [adversarial loss: -0.011972, acc: 0.000000]
24733: [discriminator loss: -0.004195, acc: 0.000000]: [adversarial loss: -0.019962, acc: 0.000000]
24734: [discriminator loss: -0.002042, acc: 0.000000]: [adversarial loss: -0.010870, acc: 0.000000]
24735: [discriminator loss: -0.004589, acc: 0.000000]: [adversarial loss: -0.010861, acc: 0.000000]
24736: [discriminator loss: -0.003390, acc: 0.000000]: [adversarial loss: -0.012571, acc: 0.000000]
24737: [discriminator loss: -0.003842, acc: 0.000000]: [adversarial loss: -0.015901, acc: 0.000000]
24738: [discriminator loss: -0.005405, acc: 0.000000]: [adversarial loss: -0.011066, acc: 0.000000]


In [19]:
from keras.models import load_model

generator = load_model("./wgan_mnist.h5")
noise = np.random.uniform(-1.0, 1.0, size=[16, 100])
gan.plot_images(generator,
                noise_input=noise,
                show=True,
                model_name="test_image"
               )

Metal device set to: Apple M2


2023-02-25 11:40:56.673930: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-25 11:40:56.674151: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


NameError: name 'plot_images' is not defined